### 實際回測狀況

In [25]:
import ffn
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import numpy as np
import base64
from io import BytesIO
from matplotlib import rcParams
rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Arial Unicode MS']  # 微軟正黑體、黑體、蘋果字型等
rcParams['axes.unicode_minus'] = False  # 解決負號顯示為方框的問題

In [26]:
df = pd.read_pickle('Backtest_Data.pkl')
df['年月'] = df['年月'].astype(int)
df = df[df['年月'] >= 201012]
df['報酬年月'] = (pd.to_datetime(df['年月'].astype(str), format='%Y%m') + pd.DateOffset(months=1)).dt.strftime('%Y%m').astype(int)
df.head(3)

,證券代碼,公司名稱,年月,月底收盤價(元),月底市值(百萬元),股價淨值比,股利殖利率,調整後股價淨值比,該月均價(元),該月均量,過去12個月報酬,下個月報酬率,報酬年月
9244,1101,台泥,201012,12.61,121103,1.48,5.49,18.6628,32.800,12555.229,4.0328,-2.7907,201101
9245,1102,亞泥,201012,14.09,99179,1.31,6.51,18.4579,32.015,7256.677,1.4779,-0.3109,201101
9246,1103,嘉泥,201012,10.11,12594,0.80,2.33,8.0880,16.625,2499.752,-1.7919,0.5990,201101


### 回測架構
>  1. 第一層選股：收盤價大於10元，避免雞蛋水餃股。
>
>  2. 第二層選股：選擇市值後20％小的股票，看好未來成長。
>
>  3. 第三層選股： 股價淨值比後 20％ 小，價值型選股。

In [27]:
results = []
for ym, group in df.groupby('報酬年月'):
    df_month = group.copy()

    # Step 1: 第一層選股 - 收盤價 > 10元，避開雞蛋水餃股
    df_step1 = df_month[df_month['月底收盤價(元)'] > 10]
    if df_step1.empty:
        results.append({'報酬年月': ym, '平均報酬率': 0})
        continue

    # Step 2: 第二層選股 - 市值後 20%，選市值小的公司
    bottom_20_mv = df_step1['月底市值(百萬元)'].quantile(0.20)
    df_step2 = df_step1[df_step1['月底市值(百萬元)'] <= bottom_20_mv]
    if df_step2.empty:
        results.append({'報酬年月': ym, '平均報酬率': 0})
        continue

    # Step 3: 第三層選股 - 股價淨值比後 20% 小，價值型選股
    bottom_20_pb = df_step2['調整後股價淨值比'].quantile(0.20)
    df_step3 = df_step2[df_step2['調整後股價淨值比'] <= bottom_20_pb]
    if df_step3.empty:
        results.append({'報酬年月': ym, '平均報酬率': 0})
        continue

    # Step 4: 計算平均報酬
    avg_return = (df_step3['下個月報酬率'].mean())/100
    results.append({'報酬年月': ym, '平均報酬率': avg_return})

# 建立結果 DataFrame
df_result = pd.DataFrame(results)

# 顯示結果
df_result

,報酬年月,平均報酬率
0,201101,0.070278
1,201102,-0.052240
2,201103,-0.031009
3,201104,0.014164
4,201105,-0.043054
...,...,...
164,202409,0.008745
165,202410,0.006317
166,202411,-0.010406
167,202412,-0.032699


### 回測結果呈現

In [28]:
# Step 1: 轉換年月為 datetime
df_result['報酬年月'] = pd.to_datetime(df_result['報酬年月'].astype(str), format='%Y%m')
df_result.set_index('報酬年月', inplace=True)

In [29]:
# Step 2: 提取報酬率 Series
returns = df_result['平均報酬率'].copy()

# Step 3: 轉換為價格指數（初始100）
price = (1 + returns).cumprod() * 100
df_price = price.to_frame(name='Portfolio')
stats = df_price.calc_stats()

# Step 4: ffn 績效分析
# 將 ffn 統計轉成 DataFrame
df_stats = stats.stats  # 已經是 DataFrame，直接拿來用
html_table = df_stats.to_html(border=0)
html_stats = df_stats.to_html(border=0)

In [30]:
# Step 5: 畫圖（累積報酬）
plt.figure(figsize=(10, 4))
df_price.plot(title='Cumulative Return', legend=False)
plt.tight_layout()
buf = BytesIO()
plt.savefig(buf, format='png')
buf.seek(0)
encoded_cumret = base64.b64encode(buf.read()).decode('utf-8')
plt.close()

# Step 6: 畫熱力圖
heatmap_data = returns.to_frame(name="return")
heatmap_data['Year'] = heatmap_data.index.year
heatmap_data['Month'] = heatmap_data.index.month
pivot_table = heatmap_data.pivot(index='Year', columns='Month', values='return')

plt.figure(figsize=(10, 5))
sns.heatmap(pivot_table, annot=True, fmt=".2%", cmap="RdYlGn", center=0)
plt.title("Monthly Return Heatmap")
plt.tight_layout()
buf = BytesIO()
plt.savefig(buf, format='png')
buf.seek(0)
encoded_heatmap = base64.b64encode(buf.read()).decode('utf-8')
plt.close()

# Step 7: 回撤圖
df_dd = df_price / df_price.cummax() - 1

plt.figure(figsize=(16, 4))
df_dd.plot(title='Drawdown', legend=False, color='red')
plt.ylabel("回撤幅度 (%)")
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.xlabel("報酬年月")
plt.tight_layout()
buf = BytesIO()
plt.savefig(buf, format='png')
buf.seek(0)
encoded_drawdown = base64.b64encode(buf.read()).decode('utf-8')
plt.close()

# Step 8: 組合 HTML
html = f"""
<html>
<head><title>Investment Performance Report</title></head>
<body>
    <h1>Investment Performance Report</h1>
    <h2>Performance Summary</h2>
    {html_stats}

    <h2>Cumulative Return</h2>
    <img src="data:image/png;base64,{encoded_cumret}" />

    <h2>Monthly Return Heatmap</h2>
    <img src="data:image/png;base64,{encoded_heatmap}" />

    <h2>Drawdown</h2>
    <img src="data:image/png;base64,{encoded_drawdown}" />
</body>
</html>
"""

# Step 9: 存成 HTML 檔
with open("performance_report.html", "w", encoding="utf-8") as f:
    f.write(html)

print("✅ 成功輸出：performance_report.html")

✅ 成功輸出：performance_report.html


<Figure size 1000x400 with 0 Axes>

<Figure size 1600x400 with 0 Axes>